<a href="https://colab.research.google.com/github/kb10241024/CSN-371_POS-Tagging/blob/main/AI_Assignment_POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: POS Tagging using HMM with Viterbi

### Mount Drive and Intial Setup

Set **path_to_project_folder** variable to the folder where Train-Corpus and Test-Corpus in present

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import os
import tarfile
import pandas as pd
from collections import Counter
from tqdm import tqdm

In [ ]:
ls

Ass2_Test_confusion_matrix.csv   Test_Result.csv
Ass2_Test_Result.csv             Test_Result_sentencewise.csv
Ass2_Train_confusion_matrix.csv  Test_Result_sentencewise.gsheet
Ass2_Training_Result.csv         Train_confusion_matrix.csv
Assigment.pdf                    Train-corups/
Results/                         Training_Result.csv
Test_confusion_matrix.csv        Training_Result_sentencewise.csv
Test-corpus/


In [ ]:
path_to_project_folder="/content/drive/My Drive/Acads/3-1/AI_Assignment/Project"
%cd $path_to_project_folder

/content/drive/My Drive/Acads/3-1/AI_Assignment/Project


In [ ]:
ls

Ass2_Test_confusion_matrix.csv   Test_Result.csv
Ass2_Test_Result.csv             Test_Result_sentencewise.csv
Ass2_Train_confusion_matrix.csv  Test_Result_sentencewise.gsheet
Ass2_Training_Result.csv         Train_confusion_matrix.csv
Assigment.pdf                    Train-corups/
Results/                         Training_Result.csv
Test_confusion_matrix.csv        Training_Result_sentencewise.csv
Test-corpus/


### Reading XML

Read XML files using ElementTree, read lines with \<s> tag, and getting words from \<w> tag,

*   **'pos'** attribute in \<w> tag has Part of Speech Tag attached with it!
*   **'hw'** attribute in \<w> tag has hard-word attatched for each word
*   parse_files function is used for parsing one file and adding knowledge to sentences,sentence_hws,sentence_tags,dictionary

In [ ]:
import xml.etree.ElementTree as ET 

In [ ]:
# For parsing one file and adding knowledge to sentences,sentence_hws,sentence_tags,dictionary
def parse_file(filename,sentences,sentence_hws,sentence_tags,dictionary):
  tree = ET.parse(filename)
  root=tree.getroot()
  flag=1
  num_sentences=0
  for sentence in root.iter('s'):
    # print(sentence.tag,sentence.attrib)
    sen=[]
    hws=[]
    tags=[]
    for child in sentence:
      # print("\t\t",child.tag,child.attrib,child.text,child.tag=='w')
      sen.append(child.text)
      if(child.tag=='w'):
        # print("\t\t\t",child.tag,child.attrib,child.text,child.tag=='w')
        hws.append(child.attrib['hw'])
        tags.append(child.attrib['c5'])
        dictionary[0].append(child.attrib['hw'])
        dictionary[1].append(child.attrib['c5'])
      # print(type(child),type(child.tag),type(child.attrib))
    if(len(sen)==0 or len(hws)==0 or len(tags)==0): continue
    sentences.append(sen)
    sentence_hws.append(hws)
    sentence_tags.append(tags)
    num_sentences+=1
  # print("Number of sentences found -",num_sentences)

### Reading Train-Corpus

*   dictionary = [[],[]], list of lists has two lists as [[hw],[tag]]
*   sentences=[], list of sentences as it is
*   sentence_hws=[] list of sentences in hw
*   sentence_tags=[] list of corresponding tags

In [ ]:
# For iterating in 
cnt = 0
dictionary = [[],[]] # [[hw],[tag]]
sentences=[] # list of sentences as it is
sentence_hws=[] # list of sentences in hw
sentence_tags=[] # list of corresponding tags
train_dir  = path_to_project_folder+"/Train-corups"
for subdir, dirs, files in tqdm(os.walk(train_dir)):
    file_cnt=0
    for file in files:
      parse_file(os.path.join(subdir, file),sentences,sentence_hws,sentence_tags,dictionary)
      # print(os.path.join(subdir, file))
      file_cnt+=1
    cnt+=file_cnt
    print("Number of files processed - ",file_cnt)
print(cnt)

0it [00:00, ?it/s]

Number of files processed -  0


2it [00:06,  3.32s/it]

Number of files processed -  26


3it [00:15,  5.03s/it]

...

Number of files processed -  32


18it [01:23,  3.50s/it]

Number of files processed -  32


19it [01:27,  4.60s/it]

Number of files processed -  31
520


In [ ]:
print(len(dictionary),len(dictionary[0]),len(dictionary[1]),len(sentences),len(sentence_hws),len(sentence_tags))

2 8904217 8904217 481637 481637 481637


In [ ]:
print(sentences[5])
print(sentence_hws[5])
print(sentence_tags[5])

['If ', 'you ', 'want ', 'Britain ', 'to ', 'stay ', 'the ', 'same ', 'then ', 'you ', 'probably ', 'wo', "n't ", 'like ', 'this ', 'manifesto', '.']
['if', 'you', 'want', 'britain', 'to', 'stay', 'the', 'same', 'then', 'you', 'probably', 'will', 'not', 'like', 'this', 'manifesto']
['CJS', 'PNP', 'VVB', 'NP0', 'TO0', 'VVI', 'AT0', 'DT0', 'AV0', 'PNP', 'AV0', 'VM0', 'XX0', 'VVI', 'DT0', 'NN1']


## Generating Model Parameters

1.   Dictionaries(unique_hws,ind_hw,unique_tags,ind_tags) for mapping
2.   Emmission Matrix(EM)
3.   Transition Matrix(TM)
4.   Tag priori probabilities


### Genrate dictionary for mapping

*   for mapping HW to index(ind_hws) and index to HW(unique_hws)
*   for mapping POS tag to index(ind_tags) and index to POS tag(unique_tags)



In [ ]:
unique_hws=list(set(dictionary[0])) # ind:hw
# Adding one extra word '<UNKNOWN>' that'll cover all unknown words
unique_hws.append('<UNKNOWN>')
unique_tags=list(set(dictionary[1])) # ind:tag
num_words=len(unique_hws)
num_tags=len(unique_tags)
print("Number of hard words - ",num_words)
print("Number of unique tags - ",num_tags)

Number of hard words -  139151
Number of unique tags -  87


In [ ]:
print(unique_tags)

['AVP', 'VVI', 'VVG', 'NN1', 'PRF', 'VHZ', 'VVZ', 'AVP-PRP', 'VBN', 'NP0-NN1', 'CJS', 'AJC', 'ITJ', 'NN1-NP0', 'PRP-AVP', 'VDI', 'VDG', 'UNC', 'CJS-AVQ', 'VVD', 'VM0', 'VBI', 'AJ0-VVD', 'PNQ', 'VHG', 'VVG-AJ0', 'VDN', 'EX0', 'AJ0', 'VHB', 'NN1-VVB', 'DPS', 'AT0', 'NP0', 'CJS-PRP', 'AV0', 'VVB-NN1', 'VVN-VVD', 'VHI', 'VVB', 'ORD', 'PRP-CJS', 'VVD-VVN', 'TO0', 'ZZ0', 'AJ0-NN1', 'PNX', 'AVQ', 'PNI-CRD', 'VDZ', 'VBG', 'PNI', 'VBZ', 'VHN', 'CRD-PNI', 'CJC', 'VDD', 'POS', 'AJ0-VVN', 'VHD', 'CJT-DT0', 'DT0', 'DT0-CJT', 'DTQ', 'CRD', 'VDB', 'NN1-AJ0', 'PNP', 'VVD-AJ0', 'VVG-NN1', 'AJ0-VVG', 'VBB', 'NN1-VVG', 'AJS', 'PRP', 'NN0', 'AV0-AJ0', 'VVZ-NN2', 'VBD', 'AVQ-CJS', 'NN2', 'NN2-VVZ', 'XX0', 'CJT', 'VVN-AJ0', 'VVN', 'AJ0-AV0']


In [ ]:
ind_hws={} # hw:ind
ind_tags={} # tag:ind
for i,tag in enumerate(unique_tags):
  ind_tags[tag]=i
for i,hw in enumerate(unique_hws):
  ind_hws[hw]=i

In [ ]:
print("Number of hard words - ",len(ind_hws))
print("Number of unique tags - ",len(ind_tags))

Number of hard words -  139151
Number of unique tags -  87


In [ ]:
print(ind_tags)
print(unique_tags)
# print(unique_tags[ind_tags['ADJ']])

{'AVP': 0, 'VVI': 1, 'VVG': 2, 'NN1': 3, 'PRF': 4, 'VHZ': 5, 'VVZ': 6, 'AVP-PRP': 7, 'VBN': 8, 'NP0-NN1': 9, 'CJS': 10, 'AJC': 11, 'ITJ': 12, 'NN1-NP0': 13, 'PRP-AVP': 14, 'VDI': 15, 'VDG': 16, 'UNC': 17, 'CJS-AVQ': 18, 'VVD': 19, 'VM0': 20, 'VBI': 21, 'AJ0-VVD': 22, 'PNQ': 23, 'VHG': 24, 'VVG-AJ0': 25, 'VDN': 26, 'EX0': 27, 'AJ0': 28, 'VHB': 29, 'NN1-VVB': 30, 'DPS': 31, 'AT0': 32, 'NP0': 33, 'CJS-PRP': 34, 'AV0': 35, 'VVB-NN1': 36, 'VVN-VVD': 37, 'VHI': 38, 'VVB': 39, 'ORD': 40, 'PRP-CJS': 41, 'VVD-VVN': 42, 'TO0': 43, 'ZZ0': 44, 'AJ0-NN1': 45, 'PNX': 46, 'AVQ': 47, 'PNI-CRD': 48, 'VDZ': 49, 'VBG': 50, 'PNI': 51, 'VBZ': 52, 'VHN': 53, 'CRD-PNI': 54, 'CJC': 55, 'VDD': 56, 'POS': 57, 'AJ0-VVN': 58, 'VHD': 59, 'CJT-DT0': 60, 'DT0': 61, 'DT0-CJT': 62, 'DTQ': 63, 'CRD': 64, 'VDB': 65, 'NN1-AJ0': 66, 'PNP': 67, 'VVD-AJ0': 68, 'VVG-NN1': 69, 'AJ0-VVG': 70, 'VBB': 71, 'NN1-VVG': 72, 'AJS': 73, 'PRP': 74, 'NN0': 75, 'AV0-AJ0': 76, 'VVZ-NN2': 77, 'VBD': 78, 'AVQ-CJS': 79, 'NN2': 80, 'NN2-VVZ':

In [ ]:
tag_distribution=dict(Counter(dictionary[1]))
# tag_distribution['<START>']=tag_distribution['<END>'] =0
print(tag_distribution)

{'VBB': 50054, 'VVN': 181699, 'DT0': 190277, 'NN1': 1342455, 'VM0': 118146, 'XX0': 59497, 'VBI': 54663, 'DTQ': 53463, 'PNP': 359829, 'VVB': 86977, 'VDZ': 4872, 'VVI': 212370, 'AJ0': 623211, 'NN2': 503345, 'AV0': 380844, 'PRP-AVP': 22833, 'AT0': 839312, 'PRF': 271444, 'VVZ': 71610, 'PRP': 727447, 'VHZ': 30014, 'TO0': 149190, 'VDN': 2540, 'NP0': 447927, 'CJS': 104825, 'CJC': 315391, 'AJC': 19142, 'PNX': 11821, 'DPS': 134247, 'AVP': 58069, 'VVG': 111607, 'VVG-AJ0': 11875, 'NN0': 52094, 'AJ0-NN1': 43447, 'ORD': 37185, 'DT0-CJT': 4132, 'VDB': 10595, 'CJT': 58175, 'POS': 61092, 'CRD': 150189, 'VBZ': 104836, 'PNQ': 23468, 'VVN-AJ0': 9921, 'AVQ': 18968, 'VDI': 5576, 'NN2-VVZ': 10856, 'AJ0-AV0': 6247, 'VVG-NN1': 10108, 'NN1-VVB': 48571, 'AJ0-VVG': 5200, 'NN1-AJ0': 22883, 'VVB-NN1': 18860, 'AJS': 11034, 'VHB': 26496, 'EX0': 20842, 'NN1-VVG': 6675, 'VVN-VVD': 19169, 'VBN': 23960, 'CJS-PRP': 4769, 'AJ0-VVD': 2615, 'VVD': 179769, 'UNC': 11345, 'AV0-AJ0': 7160, 'VHD': 39501, 'VHI': 17068, 'VBD': 111

In [ ]:
# x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_tags= {k: v for k, v in sorted(tag_distribution.items(), key=lambda item: item[1],reverse=True)}

In [ ]:
print(sorted_tags)

{'NN1': 1342455, 'AT0': 839312, 'PRP': 727447, 'AJ0': 623211, 'NN2': 503345, 'NP0': 447927, 'AV0': 380844, 'PNP': 359829, 'CJC': 315391, 'PRF': 271444, 'VVI': 212370, 'DT0': 190277, 'VVN': 181699, 'VVD': 179769, 'CRD': 150189, 'TO0': 149190, 'DPS': 134247, 'VM0': 118146, 'VBD': 111774, 'VVG': 111607, 'VBZ': 104836, 'CJS': 104825, 'VVB': 86977, 'VVZ': 71610, 'POS': 61092, 'XX0': 59497, 'CJT': 58175, 'AVP': 58069, 'VBI': 54663, 'DTQ': 53463, 'NN0': 52094, 'VBB': 50054, 'NN1-VVB': 48571, 'AJ0-NN1': 43447, 'VHD': 39501, 'ORD': 37185, 'NP0-NN1': 34543, 'VHZ': 30014, 'VHB': 26496, 'VBN': 23960, 'PNI': 23655, 'PNQ': 23468, 'NN1-AJ0': 22883, 'PRP-AVP': 22833, 'EX0': 20842, 'VVN-VVD': 19169, 'AJC': 19142, 'AVQ': 18968, 'VVB-NN1': 18860, 'VVD-VVN': 18681, 'VHI': 17068, 'NN1-NP0': 14818, 'VVG-AJ0': 11875, 'PNX': 11821, 'UNC': 11345, 'AJS': 11034, 'NN2-VVZ': 10856, 'VDB': 10595, 'VVG-NN1': 10108, 'VDD': 10104, 'VVN-AJ0': 9921, 'VBG': 8015, 'AVP-PRP': 7379, 'CJT-DT0': 7293, 'AV0-AJ0': 7160, 'NN1-VV

In [ ]:
len(sorted_tags)

87

### Create Emission Matrix
Emission matrix(EM[num_tags,num_words]) shows the probability, given a tag(in row), what is the probabilty of a given word(in columns) 

In [ ]:
def create_emission_matrix(dictionary,num_tags,num_words,tag_distribution):
  # return emission matrix(EM[num_tags*num_words]) which shows the probability of given a tag(in row), what is the probabilty of a given word(in columns) 
  EM = np.zeros((num_tags,num_words))
  for hw,tag in tqdm(zip(dictionary[0],dictionary[1])):
    EM[ind_tags[tag]][ind_hws[hw]]+=1
  for i in range(num_tags):
    EM[i]/=tag_distribution[unique_tags[i]]
  # Adding small probabilities to avoid zero, Last column corresponds to '<UNKNOWN>' keyword 
  EM[:,-1]=1e-12
  return EM

In [ ]:
EM=create_emission_matrix(dictionary,num_tags,num_words,tag_distribution)

8904217it [00:10, 830205.28it/s]


In [ ]:
print(EM.shape)

(10, 139151)


In [ ]:
print(EM[:,-1])

[1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12 1.e-12]


In [ ]:
for row in EM:
  print(np.sum(row))

1.000000000001
1.0000000000009999
1.000000000001
1.000000000001
1.000000000001
1.000000000001
1.000000000001
1.000000000001
1.0000000000009999
1.0000000000009999


### Create Transition Matrix

Transition matrix, TM[num_tags,num_tags] shows the probability of next tag given current tag


In [ ]:
def create_transition_matrix(sentence_tags,num_tags,ind_tags):
  # return transition matrix, TM[num_tags*num_tags] which shows the probability of next tag given current tag
  TM=np.zeros((num_tags,num_tags))
  for sen in tqdm(sentence_tags):
    prev='<START>'
    for tag in sen:
      if(prev=='<START>'):
        prev=tag
        continue
      TM[ind_tags[prev]][ind_tags[tag]]+=1
      prev=tag
  for row in range(TM.shape[0]):
    if(np.sum(TM[row])!=0):
      TM[row]/=np.sum(TM[row])
    # print(TM[row])
  return TM

In [ ]:
# demo=sentence_tags[:2]
# print(demo)
TM=create_transition_matrix(sentence_tags,num_tags,ind_tags)

100%|██████████| 481637/481637 [00:06<00:00, 73193.17it/s]


In [ ]:
print(TM.shape)
for row in TM:
  print(np.sum(row))

(10, 10)
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
0.9999999999999999


In [ ]:
print(TM)

[[1.02771941e-01 1.96973038e-01 2.83196734e-01 6.25661609e-02
  3.62882530e-04 5.25277475e-03 1.55991371e-01 4.55046674e-02
  6.99922211e-02 7.73882081e-02]
 [1.58965607e-02 1.05583309e-01 4.57796784e-02 6.24164613e-02
  1.28484174e-04 2.74802719e-03 9.75472946e-02 6.36150077e-01
  1.81498279e-02 1.56002800e-02]
 [1.60004996e-01 1.03793990e-01 1.92005202e-01 4.40857806e-02
  4.32194099e-04 2.17881337e-03 1.91619928e-01 7.52123467e-02
  9.35092244e-02 1.37157524e-01]
 [6.98346648e-02 1.40657099e-01 1.56681581e-01 2.08502086e-02
  3.83795740e-04 2.26237489e-03 5.05337791e-02 2.16056802e-01
  2.02567392e-01 1.40172304e-01]
 [8.18936504e-02 6.28225264e-02 1.05003365e-01 7.87525241e-02
  1.12407449e-01 6.05788647e-03 4.30783038e-02 1.96544761e-01
  2.80233341e-01 3.32061925e-02]
 [2.07666604e-02 2.18326431e-01 3.52386107e-02 3.06852410e-02
  3.29442771e-04 2.93086408e-02 5.66759224e-02 5.84243223e-01
  8.36549322e-03 1.60603351e-02]
 [1.33770909e-02 1.54425294e-01 1.56207315e-01 2.47436534e

### Creating Tag priori probabilities

In [ ]:
def create_priori_probabilities(tag_distribution,ind_tags):
  # return an array of tag priori probabilities,TPP[num_tags,] based on frequency
  num_tags=len(ind_tags)
  total_tags=sum([tag_distribution[key] for key in tag_distribution])
  TPP=np.zeros(num_tags)
  for tag,freq in  tag_distribution.items():
    TPP[ind_tags[tag]]=freq/total_tags
  return TPP

In [ ]:
TPP=create_priori_probabilities(tag_distribution,ind_tags)

In [ ]:
print(TPP.shape)
print(np.sum(TPP))
print(TPP)

(10,)
0.9999999999999999
[0.05994497 0.1228528  0.17577649 0.05566172 0.00064587 0.01014239
 0.13207776 0.27800052 0.07064012 0.09425736]


## Model

### Viterbi HMM Algorithm

In [ ]:
def viterbi(inp, TM, EM, TPP=None):
    """
    Return the MAP estimate of state trajectory of Hidden Markov Model.

    Parameters
    ----------
    inp : array (T,)
        Observation state sequence. int dtype.
    TM : array (K, K)
        State transition matrix. See HiddenMarkovModel.state_transition  for
        details.
    EM : array (K, M)
        Emission matrix. See HiddenMarkovModel.emission for details.
    TPP: optional, (K,)
        Initial state probabilities: TPP[i] is the probability x[0] == i. If
        None, uniform initial distribution is assumed (TPP[:] == 1/K).

    Returns
    -------
    out: array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory,
        conditioned on observation sequence inp under the model parameters TM, EM,
        TPP.
    T1: array (K, T)
        the probability of the most likely path so far
    T2: array (K, T)
        the x_j-1 of the most likely path so far
    """
    # Cardinality of the state space
    K = TM.shape[0]
    # Initialize the priors with default (uniform dist) if not given by caller
    TPP = TPP if TPP is not None else np.full(K, 1 / K)
    T = len(inp)
    T1 = np.empty((K, T), 'd')
    T2 = np.empty((K, T), 'B')

    # Initilaize the tracking tables from first observation
    T1[:, 0] = TPP * EM[:, inp[0]]
    T2[:, 0] = 0

    # Iterate throught the observations updating the tracking tables
    for i in range(1, T):
        T1[:, i] = np.max(T1[:, i - 1] * TM.T * EM[np.newaxis, :, inp[i]].T, 1)
        T2[:, i] = np.argmax(T1[:, i - 1] * TM.T, 1)

    # Build the output, optimal model trajectory
    out= np.empty(T, 'B')
    out[-1] = np.argmax(T1[:, T - 1])
    for i in reversed(range(1, T)):
        out[i - 1] = T2[out[i], i]

    return out, T1, T2

### Testing Viterbi

predict function returns sequence for POS tags, given a sentence

In [ ]:
def predict(q,TM,EM,TPP,ind_hws):
  input=np.zeros(len(q),dtype=np.int32)
  for i,hw in enumerate(q):
    # Out of vocabulary words is marked as '<UNKNOWN>' 
    if(not (hw in unique_hws)):
      hw='<UNKNOWN>'
    input[i]=int(ind_hws[hw])
  output,T1,T2=viterbi(input, TM, EM, TPP)
  ans=[]
  for i in output:
    ans.append(unique_tags[i])
  return ans

In [ ]:
q=sentence_hws[7]
a=sentence_tags[7]
print(q)
print(a)

['britain', 'have', 'a', 'clear', 'choice', 'at', 'this', 'election']
['SUBST', 'VERB', 'ART', 'ADJ', 'SUBST', 'PREP', 'ADJ', 'SUBST']


In [ ]:
# q=[]
ans=predict(q,TM,EM,TPP,ind_hws)
print(ans)

['SUBST', 'VERB', 'ART', 'ADJ', 'SUBST', 'PREP', 'ADJ', 'SUBST']


# Running Model and Analysis Parts

## For Training set

### Running Model and Generating prediction

In [ ]:
prediction_tags=[]
for sen in tqdm(sentence_hws):
  ans=predict(sen,TM,EM,TPP,ind_hws)
  prediction_tags.append(ans)

100%|██████████| 481637/481637 [02:36<00:00, 3075.88it/s]


In [ ]:
print(sentence_tags[:5])
print(prediction_tags[:5])

["['VERB', 'VERB', 'ADJ', 'SUBST', 'VERB', 'ADV', 'VERB', 'PRON', 'PRON', 'VERB']", "['SUBST']", "['ADJ', 'SUBST', 'VERB', 'ADV', 'VERB', 'ADJ', 'SUBST', 'ADV', 'PREP', 'ART', 'SUBST']", "['PRON', 'VERB', 'ADV', 'VERB', 'ADJ', 'SUBST', 'SUBST', 'PREP', 'SUBST']", "['PRON', 'ADV', 'VERB', 'ART', 'SUBST', 'ART', 'SUBST', 'PREP', 'PRON', 'ADJ', 'SUBST', 'VERB', 'VERB', 'PREP', 'VERB', 'VERB', 'PREP', 'SUBST', 'PREP', 'VERB']"]
[['VERB', 'VERB', 'ADJ', 'SUBST', 'VERB', 'ADV', 'VERB', 'PRON', 'PRON', 'VERB'], ['SUBST'], ['ADJ', 'SUBST', 'VERB', 'ADV', 'VERB', 'ADJ', 'SUBST', 'ADV', 'PREP', 'ART', 'SUBST'], ['PRON', 'VERB', 'ADV', 'VERB', 'ADJ', 'SUBST', 'SUBST', 'PREP', 'SUBST'], ['PRON', 'ADV', 'VERB', 'ART', 'SUBST', 'ART', 'SUBST', 'PREP', 'PRON', 'ADJ', 'SUBST', 'VERB', 'VERB', 'PREP', 'VERB', 'VERB', 'PREP', 'SUBST', 'PREP', 'VERB']]


In [ ]:
print(len(sentence_tags),len(prediction_tags))

481637 481637


### Saving Training Result

In [ ]:
result={'Sentence Tags':sentence_tags,'Predicted Tags':prediction_tags}

In [ ]:
ls

Assigment.pdf                 Test_Result_sentencewise.gsheet
Results/                      Train-corups/
Test-corpus/                  Training_Result.csv
Test_Result.csv               Training_Result_sentencewise.csv
Test_Result_sentencewise.csv


In [ ]:
df =  pd.DataFrame(result)
df.to_csv("Training_Result_sentencewise.csv")

In [ ]:
taglist=[]
predictions=[]

In [ ]:
false_count=0
for sen,pred in zip(sentence_tags,prediction_tags):
  for tag,prediction in zip(sen,pred):
    taglist.append(tag)
    predictions.append(prediction)
    if(prediction!=tag): false_count+=1

In [ ]:
print(len(taglist),len(predictions),false_count)
print("Accuracy is ",(1-false_count/len(taglist)))

8904217 8904217 326005
Accuracy is  0.963387572427761


In [ ]:
df2=pd.DataFrame({'Tags':taglist,'Prediction':predictions})
df2.to_csv("Training_Result.csv")

### Confusion Matrix and report for Training

In [ ]:
# Importing the dependancies
from sklearn import metrics

# The confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
train_cf_matrix=metrics.confusion_matrix(y_true=taglist,y_pred=predictions, labels=unique_tags)

In [ ]:
print(train_cf_matrix)

[[ 473694      55       0   11246       0      55    6504    4069       1
        0]
 [    359 1486648      28    1240      78   70143    5707     876      20
       53]
 [     89      12  624571       0       0     142    3996     185       0
        0]
 [   8194    1112       0 1161059       1     229     289    5160       5
        0]
 [     20      39       0      14    4706     224      56      38       8
      646]
 [   1579   55774     608     446      68 2377335   34659    1846    1413
     1649]
 [  13027    6087     882     492      12   13571 1041684   18146       7
        0]
 [  12684     742    3088   17083       5    1996   17963  480099       0
      103]
 [     98      19      16      18       5     552      27       6   89555
       14]
 [    225       6       1       2      25     144       1      23       0
   838861]]


In [ ]:
# Printing the precision and recall, among other metrics
train_report=metrics.classification_report(y_true=taglist, y_pred=predictions, labels=unique_tags)

In [ ]:
print(type(train_report))
print(train_report)

<class 'str'>
              precision    recall  f1-score   support

        CONJ       0.93      0.96      0.94    495624
        VERB       0.96      0.95      0.95   1565152
        PRON       0.99      0.99      0.99    628995
        PREP       0.97      0.99      0.98   1176049
      INTERJ       0.96      0.82      0.88      5751
       SUBST       0.96      0.96      0.96   2475377
         ADJ       0.94      0.95      0.94   1093908
         ADV       0.94      0.90      0.92    533763
         UNC       0.98      0.99      0.99     90310
         ART       1.00      1.00      1.00    839288

    accuracy                           0.96   8904217
   macro avg       0.96      0.95      0.96   8904217
weighted avg       0.96      0.96      0.96   8904217



In [ ]:
report_df=pd.DataFrame(train_cf_matrix,index=unique_tags,columns=unique_tags)

In [ ]:
report_df

,CONJ,VERB,PRON,PREP,INTERJ,SUBST,ADJ,ADV,UNC,ART
CONJ,473694,55,0,11246,0,55,6504,4069,1,0
VERB,359,1486648,28,1240,78,70143,5707,876,20,53
PRON,89,12,624571,0,0,142,3996,185,0,0
PREP,8194,1112,0,1161059,1,229,289,5160,5,0
INTERJ,20,39,0,14,4706,224,56,38,8,646
SUBST,1579,55774,608,446,68,2377335,34659,1846,1413,1649
ADJ,13027,6087,882,492,12,13571,1041684,18146,7,0
ADV,12684,742,3088,17083,5,1996,17963,480099,0,103
UNC,98,19,16,18,5,552,27,6,89555,14
ART,225,6,1,2,25,144,1,23,0,838861


In [ ]:
report_df.to_csv('Train_confusion_matrix.csv')

## For Test dataset

### Read Test-corpus


In [ ]:
ls

Assigment.pdf  Train-corups/        Training_Result_sentencewise.csv
Test-corpus/   Training_Result.csv


In [ ]:
cnt = 0
test_dictionary = [[],[]] # [[hw],[tag]]
test_sentences=[] # list of sentences as it is
test_sentence_hws=[] # list of sentences in hw
test_sentence_tags=[] # list of corresponding tags
test_dir  = path_to_project_folder+"/Test-corpus"
for subdir, dirs, files in os.walk(test_dir):
    file_cnt=0
    for file in files:
      parse_file(os.path.join(subdir, file),test_sentences,test_sentence_hws,test_sentence_tags,test_dictionary)
      # print(os.path.join(subdir, file))
      file_cnt+=1
    cnt+=file_cnt
    print("Number of files processed - ",file_cnt)
print(cnt)

Number of files processed -  0
Number of files processed -  10
Number of files processed -  6
Number of files processed -  24
Number of files processed -  26
Number of files processed -  26
Number of files processed -  23
115


In [ ]:
print(len(test_dictionary),len(test_dictionary[0]),len(test_dictionary[1]),len(test_sentences),len(test_sentence_hws),len(test_sentence_tags))
print(test_sentences[0])
print(test_sentence_hws[0])
print(test_sentence_tags[0])

2 3536068 3536068 199743 199743 199743
['‘', 'The ', 'ideal ', 'introduction ', 'to ', 'discussion ', 'and ', 'debate ', 'at ', 'parish', ', ', 'deanery ', 'and ', 'synod ', 'level', '.']
['the', 'ideal', 'introduction', 'to', 'discussion', 'and', 'debate', 'at', 'parish', 'deanery', 'and', 'synod', 'level']
['ART', 'ADJ', 'SUBST', 'PREP', 'SUBST', 'CONJ', 'SUBST', 'PREP', 'SUBST', 'SUBST', 'CONJ', 'SUBST', 'SUBST']


### Running Model and Generating Test Predictions

In [ ]:
test_prediction_tags=[]
for sen in tqdm(test_sentence_hws):
  if(len(sen)==0):
    continue
  ans=predict(sen,TM,EM,TPP,ind_hws)
  test_prediction_tags.append(ans)

In [ ]:
print(len(test_sentence_tags),len(test_prediction_tags))

199743 199743


### Saving Test Predictions

In [ ]:
test_result={'Sentence Tags':test_sentence_tags,'Predicted Tags':test_prediction_tags}
test_df =  pd.DataFrame(test_result)
test_df.to_csv("Test_Result_sentencewise.csv")

In [ ]:
test_taglist=[]
test_predictions=[]

In [ ]:
test_false_count=0
for sen,pred in zip(test_sentence_tags,test_prediction_tags):
  for tag,prediction in zip(sen,pred):
    test_taglist.append(tag)
    test_predictions.append(prediction)
    if(prediction!=tag): test_false_count+=1

In [ ]:
print(len(test_taglist),len(test_predictions),test_false_count)
print("Accuracy is ",(1-test_false_count/len(test_taglist)))

3536068 3536068 158574
Accuracy is  0.9551552741632796


In [ ]:
test_df2=pd.DataFrame({'Tags':test_taglist,'Prediction':test_predictions})
test_df2.to_csv("Test_Result.csv")

### Confusion Matrix and report for test-corpus

In [ ]:
# Importing the dependancies
from sklearn import metrics

# The confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
cf_matrix=metrics.confusion_matrix(y_true=test_taglist,y_pred=test_predictions, labels=unique_tags)

In [ ]:
print(cf_matrix)

[[200582     16      0   4476      0     17   2117   2083      0      0]
 [   261 615233     24    682     11  27190   2506    290      5    215]
 [   113     14 276566      2      0     75   1815    155      0      3]
 [  2911    446      0 461406      0     91    128   2306      0      8]
 [     0     32      2      4   3100    128    102     15      1    418]
 [  1067  25529   1566   4190     45 852288  15561    894    413   4687]
 [  5450   3160    568    876      8   8280 401367   7556      6   2052]
 [  5131    411   1442   7195      3    976   7596 207245      0    102]
 [   123    333     54    953      9   2247    471     23  21345    731]
 [   122      3      0      1     17     53      0      8      0 338362]]


In [ ]:
# Printing the precision and recall, among other metrics
report=metrics.classification_report(y_true=test_taglist, y_pred=test_predictions, labels=unique_tags)

In [ ]:
print(type(report))
print(report)

<class 'str'>
              precision    recall  f1-score   support

        CONJ       0.93      0.96      0.94    209291
        VERB       0.95      0.95      0.95    646417
        PRON       0.99      0.99      0.99    278743
        PREP       0.96      0.99      0.97    467296
      INTERJ       0.97      0.82      0.89      3802
       SUBST       0.96      0.94      0.95    906240
         ADJ       0.93      0.93      0.93    429323
         ADV       0.94      0.90      0.92    230101
         UNC       0.98      0.81      0.89     26289
         ART       0.98      1.00      0.99    338566

    accuracy                           0.96   3536068
   macro avg       0.96      0.93      0.94   3536068
weighted avg       0.96      0.96      0.95   3536068



In [ ]:
test_report_df=pd.DataFrame(cf_matrix,index=unique_tags,columns=unique_tags)

In [ ]:
test_report_df

,CONJ,VERB,PRON,PREP,INTERJ,SUBST,ADJ,ADV,UNC,ART
CONJ,200582,16,0,4476,0,17,2117,2083,0,0
VERB,261,615233,24,682,11,27190,2506,290,5,215
PRON,113,14,276566,2,0,75,1815,155,0,3
PREP,2911,446,0,461406,0,91,128,2306,0,8
INTERJ,0,32,2,4,3100,128,102,15,1,418
SUBST,1067,25529,1566,4190,45,852288,15561,894,413,4687
ADJ,5450,3160,568,876,8,8280,401367,7556,6,2052
ADV,5131,411,1442,7195,3,976,7596,207245,0,102
UNC,123,333,54,953,9,2247,471,23,21345,731
ART,122,3,0,1,17,53,0,8,0,338362


In [ ]:
test_report_df.to_csv('Test_confusion_matrix.csv')

# Assignment 2

## Model Parameters

In [ ]:
def ass2_predict(hw,MM):
  if(hw in ind_hws.keys()):
    return unique_tags[np.argmax(MM[ind_hws[hw]])]
  else:
    return "SUBST"

In [ ]:
def create_map_matrix(dictionary,num_tags,num_words):
  # return emission matrix(EM[num_tags*num_words]) which shows the probability of given a tag(in row), what is the probabilty of a given word(in columns) 
  MM = np.zeros((num_words,num_tags))
  for hw,tag in tqdm(zip(dictionary[0],dictionary[1])):
    MM[ind_hws[hw]][ind_tags[tag]]+=1
  for i in range(num_words):
    MM[i]/=np.sum(MM[i])
  # Adding small probabilities to avoid zero, Last column corresponds to '<UNKNOWN>' keyword 
  MM[-1,:]=1e-12
  return MM

In [ ]:
MM=create_map_matrix(dictionary,num_tags,num_words)

8904217it [00:10, 822767.49it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [ ]:
MM.shape

(139151, 10)

## Train

In [ ]:
ass2_predictions=[]
for hw in tqdm(dictionary[0]):
  ass2_predictions.append(ass2_predict(hw,MM))

100%|██████████| 8904217/8904217 [00:21<00:00, 407453.62it/s]


In [ ]:
print(len(dictionary[0]),len(dictionary[1]),len(ass2_predictions))

8904217 8904217 8904217


In [ ]:
ass2_result={'True Tags':dictionary[1],'Predicted Tags':ass2_predictions}

In [ ]:
ass2_result={'True Tags':dictionary[1],'Predicted Tags':ass2_predictions}
df2=pd.DataFrame(ass2_result)
df2.to_csv("Ass2_Training_Result.csv")

In [ ]:
ls

Ass2_Training_Result.csv   Test_Result_sentencewise.csv
Assigment.pdf              Test_Result_sentencewise.gsheet
Results/                   Train_confusion_matrix.csv
Test_confusion_matrix.csv  Train-corups/
Test-corpus/               Training_Result.csv
Test_Result.csv            Training_Result_sentencewise.csv


In [ ]:
# Importing the dependancies
from sklearn import metrics

# The confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
ass2_train_cf_matrix=metrics.confusion_matrix(y_true=dictionary[1],y_pred=ass2_predictions, labels=unique_tags)

In [ ]:
ass2_report_df=pd.DataFrame(ass2_train_cf_matrix,index=unique_tags,columns=unique_tags)

In [ ]:
ass2_report_df

,ADV,ADJ,VERB,CONJ,INTERJ,UNC,PREP,SUBST,PRON,ART
ADV,471613,15053,91,11215,1,0,28112,3241,4277,160
ADJ,25960,1015952,6199,21628,6,0,1433,22323,407,0
VERB,997,10547,1437718,39,18,23,3117,112518,52,123
CONJ,7401,0,0,481532,0,0,6691,0,0,0
INTERJ,0,145,37,0,3978,17,2,156,0,1416
UNC,4,32,30,6,1,89418,25,763,17,14
PREP,5266,706,42,21752,27,4,1148198,54,0,0
SUBST,3415,56061,83715,519,134,155,1006,2327955,577,1840
PRON,2,6877,0,0,0,0,0,28,622088,0
ART,0,1,0,0,2,0,0,0,1,839284


In [ ]:
ass2_report_df.to_csv('Ass2_Train_confusion_matrix.csv')

In [ ]:
# Printing the precision and recall, among other metrics
ass2_train_report=metrics.classification_report(y_true=dictionary[1],y_pred=ass2_predictions, labels=unique_tags)
# print(type(train_report))
print(ass2_train_report)

              precision    recall  f1-score   support

         ADV       0.92      0.88      0.90    533763
         ADJ       0.92      0.93      0.92   1093908
        VERB       0.94      0.92      0.93   1565152
        CONJ       0.90      0.97      0.93    495624
      INTERJ       0.95      0.69      0.80      5751
         UNC       1.00      0.99      0.99     90310
        PREP       0.97      0.98      0.97   1176049
       SUBST       0.94      0.94      0.94   2475377
        PRON       0.99      0.99      0.99    628995
         ART       1.00      1.00      1.00    839288

    accuracy                           0.95   8904217
   macro avg       0.95      0.93      0.94   8904217
weighted avg       0.95      0.95      0.95   8904217



## Test

In [ ]:
ass2_test_predictions=[]
for hw in tqdm(test_dictionary[0]):
  ass2_test_predictions.append(ass2_predict(hw,MM))

100%|██████████| 3536068/3536068 [00:09<00:00, 365187.81it/s]


In [ ]:
print(len(test_dictionary[0]),len(test_dictionary[1]),len(ass2_test_predictions))

3536068 3536068 3536068


In [ ]:
ass2_test_result={'True Tags':test_dictionary[1],'Predicted Tags':ass2_test_predictions}
df2=pd.DataFrame(ass2_test_result)
df2.to_csv("Ass2_Test_Result.csv")

In [ ]:
# Importing the dependancies
from sklearn import metrics

# The confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
ass2_test_cf_matrix=metrics.confusion_matrix(y_true=test_dictionary[1],y_pred=ass2_test_predictions, labels=unique_tags)

In [ ]:
ass2_test_report_df=pd.DataFrame(ass2_test_cf_matrix,index=unique_tags,columns=unique_tags)

In [ ]:
ass2_test_report_df

,ADV,ADJ,VERB,CONJ,INTERJ,UNC,PREP,SUBST,PRON,ART
ADV,203701,6294,32,4364,0,0,11871,1786,1991,62
ADJ,10164,387462,2717,9066,1,4,615,19015,279,0
VERB,302,4377,594562,9,11,7,1351,45769,20,9
CONJ,3172,0,0,203397,0,0,2716,6,0,0
INTERJ,0,134,23,0,2655,2,0,113,0,875
UNC,5,35,12,8,8,21177,95,4938,5,6
PREP,2268,306,23,8538,12,1,456119,29,0,0
SUBST,1439,20016,32590,233,53,130,343,850451,240,745
PRON,2,3180,0,0,0,0,0,61,275500,0
ART,0,0,0,0,0,0,0,5,0,338561


In [ ]:
ass2_test_report_df.to_csv('Ass2_Test_confusion_matrix.csv')

In [ ]:
# Printing the precision and recall, among other metrics
ass2_test_report=metrics.classification_report(y_true=test_dictionary[1],y_pred=ass2_test_predictions, labels=unique_tags)
# print(type(train_report))
print(ass2_test_report)

              precision    recall  f1-score   support

         ADV       0.92      0.89      0.90    230101
         ADJ       0.92      0.90      0.91    429323
        VERB       0.94      0.92      0.93    646417
        CONJ       0.90      0.97      0.94    209291
      INTERJ       0.97      0.70      0.81      3802
         UNC       0.99      0.81      0.89     26289
        PREP       0.96      0.98      0.97    467296
       SUBST       0.92      0.94      0.93    906240
        PRON       0.99      0.99      0.99    278743
         ART       1.00      1.00      1.00    338566

    accuracy                           0.94   3536068
   macro avg       0.95      0.91      0.93   3536068
weighted avg       0.94      0.94      0.94   3536068

